In [1]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 62.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 25.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 71.7 MB/s eta 0:00:00


In [2]:
#!pip install accelerate

In [3]:
from transformers import AutoModel, AutoTokenizer
#from accelerate import Accelerator
import pandas as pd
import torch
from transformers import BertTokenizer, BertModel
from sklearn.svm import SVC
from sklearn.metrics import classification_report

In [4]:
# Load the pre-trained BERT model and tokenizer
model_name = 'bert-base-uncased'
model = AutoModel.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [5]:
def extract_features(text):
    # Tokenize the text and add special tokens for BERT
    input_ids = torch.tensor(tokenizer.encode(text, add_special_tokens=True)).unsqueeze(0)
    # Encode the input tokens with BERT and get the hidden states
    outputs = model(input_ids)
    hidden_states = outputs.last_hidden_state
    # Apply max pooling to the hidden states to get a fixed-length representation of the text
    pooled = torch.max(hidden_states, 1)[0]
    return pooled.detach().numpy()

In [6]:
#import pandas as pd

#df = pd.read_csv('/content/drive/MyDrive/augmented_tweets.csv',engine="python",error_bad_lines=False)
df = pd.read_csv('/content/drive/MyDrive/restructured_data.csv',engine="python",error_bad_lines=False)

<ipython-input-6-1cce8ac9d852>:4: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df = pd.read_csv('/content/drive/MyDrive/restructured_data.csv',engine="python",error_bad_lines=False)


In [7]:
print(df.head())

                                                Data   Stance
0  Yup The woman in the womb are a precious a tho...  AGAINST
1  Yup One of the MANY reason I changed party see...     NONE
2  your yr outofdate I thought you could be more ...     NONE
3  Your wee sister belief in both Santa Claus and...  AGAINST
4  your tweet ha made it to the cast of townie ma...     NONE


In [8]:
#df['text'] = df['target'] + ' ' + df['extended_target'] + ' ' + df['target_defintion'] + ' ' + df['tweets'] + ' ' + df['reformulated_tweets']
df['text'] = df['Data']

In [9]:
print(df['text'])

0       Yup The woman in the womb are a precious a tho...
1       Yup One of the MANY reason I changed party see...
2       your yr outofdate I thought you could be more ...
3       Your wee sister belief in both Santa Claus and...
4       your tweet ha made it to the cast of townie ma...
                              ...                        
2809    10 Holy Mary Mother of God pray for u sinner n...
2810    1 Holy Mary Mother of God pray for u sinner no...
2811    1 Corinthians 1313 So now faith hope and love ...
2812    1 Cor 1558 stand firmAlways give ourselves ful...
2813    _AnAvgJoe_ it Ruth_Blane BansalSatnam it Ruth_...
Name: text, Length: 2814, dtype: object


In [ ]:
# inputs = tokenizer(df['text'].tolist(), return_tensors='pt', padding=True, truncation=True)

# # Apply BERT to the input tensors and get the hidden states
# with torch.no_grad():
#     outputs = model(**inputs)

# # Get the hidden states tensor
# hidden_states = outputs.last_hidden_state

# # Apply max pooling to the hidden states to get a fixed-length representation of the input text
# pooled = torch.max(hidden_states, 1)[0]

# # Convert the pooled tensor to a numpy array
# features = pooled.numpy()

In [10]:
features = [extract_features(text) for text in df['text']]

In [11]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import numpy as np

In [12]:
print(len(features[3]))

1


In [13]:
X = np.array(features).reshape(len(features), -1)

In [ ]:
# nsamples, nx, ny = features.shape
# new_features = features.reshape((nsamples,nx*ny))

In [15]:
from sklearn import preprocessing

le = preprocessing.LabelEncoder()
le.fit(df.Stance)
df['categorical_label'] = le.transform(df.Stance)


In [16]:
print(df['categorical_label'].unique())

[0 2 1]


In [17]:
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import precision_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score

In [18]:
X_train, X_test, y_train, y_test = train_test_split(X, df['categorical_label'], test_size=0.2, random_state=42)

# Train an SVM classifier on the features
clf = SVC(kernel='linear')
clf.fit(X_train, y_train)

# Make predictions on the test data
y_pred = clf.predict(X_test)

# Calculate the accuracy of the predictions
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.5506216696269982


In [19]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.61      0.67      0.64       267
           1       0.49      0.42      0.45       160
           2       0.48      0.46      0.47       136

    accuracy                           0.55       563
   macro avg       0.53      0.52      0.52       563
weighted avg       0.54      0.55      0.55       563



In [20]:
from sklearn.model_selection import GridSearchCV
  
# defining parameter range
param_grid = {'C': [0.1, 1, 10, 100, 1000], 
              'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
              'kernel': ['rbf']} 
  
grid = GridSearchCV(SVC(), param_grid, refit = True, verbose = 3)
  
# fitting the model for grid search
grid.fit(X_train, y_train)

Fitting 5 folds for each of 25 candidates, totalling 125 fits
[CV 1/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.477 total time=   1.7s
[CV 2/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.478 total time=   1.6s
[CV 3/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.478 total time=   1.6s
[CV 4/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.478 total time=   1.6s
[CV 5/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.478 total time=   1.6s
[CV 1/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.477 total time=   1.7s
[CV 2/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.478 total time=   2.1s
[CV 3/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.478 total time=   2.6s
[CV 4/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.478 total time=   2.2s
[CV 5/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.478 total time=   1.6s
[CV 1/5] END .....C=0.1, gamma=0.01, kernel=rbf;, score=0.550 total time=   1.3s
[CV 2/5] END .....C=0.1, gamma=0.01, kernel=rbf

GridSearchCV(estimator=SVC(),
             param_grid={'C': [0.1, 1, 10, 100, 1000],
                         'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
                         'kernel': ['rbf']},
             verbose=3)

In [21]:

# print best parameter after tuning
print(grid.best_params_)
  
# print how our model looks after hyper-parameter tuning
print(grid.best_estimator_)

{'C': 10, 'gamma': 0.01, 'kernel': 'rbf'}
SVC(C=10, gamma=0.01)


In [22]:

grid_predictions = grid.predict(X_test)
  
# print classification report
print(classification_report(y_test, grid_predictions))


              precision    recall  f1-score   support

           0       0.69      0.75      0.72       267
           1       0.61      0.51      0.55       160
           2       0.55      0.55      0.55       136

    accuracy                           0.63       563
   macro avg       0.61      0.60      0.61       563
weighted avg       0.63      0.63      0.63       563



In [23]:
from sklearn.metrics import confusion_matrix

print(confusion_matrix(y_true=y_test, y_pred=y_pred))

[[180  46  41]
 [ 67  67  26]
 [ 50  23  63]]


In [24]:
print("Accuracy: ",accuracy_score(y_test, grid_predictions))
print("Recall Score: ",recall_score(y_test, grid_predictions, average='weighted'))
print("Precision Score: ",precision_score(y_test, grid_predictions, average='weighted'))
print("F1 Score: ",f1_score(y_test, grid_predictions, average='weighted'))

Accuracy:  0.6341030195381883
Recall Score:  0.6341030195381883
Precision Score:  0.6306571692587551
F1 Score:  0.6302971504274555


In [25]:
import pickle

with open('BERT_SVM_model_restructured.pkl','wb') as f:
    pickle.dump(grid,f)